In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn import tree
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import visualkeras
import tensorflow as tf
tf.data.experimental.enable_debug_mode()
from tensorflow.keras.callbacks import ModelCheckpoint
from tqdm.notebook import tqdm_notebook
from sklearn.tree import export_graphviz
from collections import defaultdict


In [2]:
def sequence_mining(team,opponent):
    combined_df = pd.read_csv(str(team)+'_runs.csv')
    combined_df = combined_df.replace({str(team):'same', str(opponent):'other'}, regex=True)
    
    df = pd.DataFrame()
    encoders = []

    for column in combined_df.columns[:-1]:
        le = LabelEncoder()
        encoders.append(le)
        df[column] = le.fit_transform(combined_df[column])

    df = pd.concat([df,combined_df.iloc[:,-1]], axis=1)

    undersample_len = len(df[df['class'] == 1])

    undersample_df = df[df['class'] == 0 ].sample(n=undersample_len, random_state=43)
    df = pd.concat([df[df['class'] == 1], undersample_df])

    print(df['class'].value_counts())
    
    run_events = {}
    run_max_counts = []

    for j, event in zip(range(12,112,11), range(10, 0, -1)):
        a = combined_df.iloc[:, -j:-1][combined_df['class'] == 1]

        # count occurrences of each row
        row_counts = defaultdict(int)
        for i in range(len(a)):
            row_tuple = tuple(a.iloc[i])
            row_counts[row_tuple] += 1

        # row with the maximum count
        max_count = 0
        mc_row = None
        for row, count in row_counts.items():
            if count > max_count:
                max_count = count
                mc_row = row

        # index of the row 
        mc_idx = a.apply(lambda row: tuple(row) == mc_row, axis=1).idxmax()


        print(f"Index: {mc_idx}, Max Count: {max_count}")
        run_events[event] = a.iloc[mc_idx]
        run_max_counts.append(max_count)
        print(f'{combined_df.iloc[mc_idx, -j:-1].to_frame().dropna().T}\n')

In [3]:
sequence_mining('home','away')

C:\Users\gsevr\AppData\Local\Temp\ipykernel_242584\3331803452.py:2: DtypeWarning: Columns (104) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_df = pd.read_csv(str(team)+'_runs.csv')
C:\Users\gsevr\AppData\Local\Temp\ipykernel_242584\3331803452.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column] = le.fit_transform(combined_df[column])
C:\Users\gsevr\AppData\Local\Temp\ipykernel_242584\3331803452.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column] = le.fit_transform(co

class
1    18471
0    18471
Name: count, dtype: int64
Index: 16388, Max Count: 562
          Fouler10
16388  fouler_same

Index: 16418, Max Count: 108
           Rebounder9 ReboundType9     Fouler10
16418  rebounder_same    defensive  fouler_same

Index: 16446, Max Count: 53
            Rebounder8 ReboundType8 ShotDist9       Shooter9     Rebounder10   
16446  rebounder_other    defensive       3pt  shooter_other  rebounder_same  \

      ReboundType10  
16446     defensive  

Index: 16446, Max Count: 30
      ShotDist7      Shooter7       Rebounder8 ReboundType8 ShotDist9   
16446       3pt  shooter_same  rebounder_other    defensive       3pt  \

            Shooter9     Rebounder10 ReboundType10  
16446  shooter_other  rebounder_same     defensive  

Index: 16418, Max Count: 16
      ShotDist6      Shooter6       Rebounder7 ReboundType7 ShotDist8   
16418       3pt  shooter_same  rebounder_other    defensive       3pt  \

            Shooter8      Rebounder9 ReboundType9     Fouler1

In [4]:
sequence_mining('away', 'home')

C:\Users\gsevr\AppData\Local\Temp\ipykernel_242584\3331803452.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column] = le.fit_transform(combined_df[column])
C:\Users\gsevr\AppData\Local\Temp\ipykernel_242584\3331803452.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column] = le.fit_transform(combined_df[column])
C:\Users\gsevr\AppData\Local\Temp\ipykernel_242584\3331803452.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor 

class
1    16497
0    16497
Name: count, dtype: int64
Index: 3, Max Count: 4457
      Fouler10
3  fouler_same

Index: 6, Max Count: 978
  ShotDist9       Shooter9     Fouler10
6     close  shooter_other  fouler_same

Index: 17, Max Count: 423
         Rebounder8 ReboundType8 ShotDist9       Shooter9     Rebounder10   
17  rebounder_other    defensive       3pt  shooter_other  rebounder_same  \

   ReboundType10  
17     defensive  

Index: 171, Max Count: 194
    ShotDist7      Shooter7       Rebounder8 ReboundType8 ShotDist9   
171       3pt  shooter_same  rebounder_other    defensive       3pt  \

          Shooter9     Rebounder10 ReboundType10  
171  shooter_other  rebounder_same     defensive  

Index: 73, Max Count: 54
         Rebounder6 ReboundType6       Fouler7 FreeThrowShooter8   
73  rebounder_other    defensive  fouler_other          ft_other  \

   FreeThrowShooter9     Fouler10  
73          ft_other  fouler_same  

Index: 73, Max Count: 24
   ShotDist5      Shooter5    

In [5]:
combined_df = pd.read_csv('home_runs.csv')
combined_df

C:\Users\gsevr\AppData\Local\Temp\ipykernel_242584\2790771757.py:1: DtypeWarning: Columns (104) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_df = pd.read_csv('home_runs.csv')


,ShotDist1,TimeoutTeam1,Substitution1,Shooter1,Rebounder1,Blocker1,Fouler1,ReboundType1,ViolationPlayer1,FreeThrowShooter1,TurnoverPlayer1,ShotDist2,TimeoutTeam2,Substitution2,Shooter2,Rebounder2,Blocker2,Fouler2,ReboundType2,ViolationPlayer2,FreeThrowShooter2,TurnoverPlayer2,ShotDist3,TimeoutTeam3,Substitution3,Shooter3,Rebounder3,Blocker3,Fouler3,ReboundType3,ViolationPlayer3,FreeThrowShooter3,TurnoverPlayer3,ShotDist4,TimeoutTeam4,Substitution4,Shooter4,Rebounder4,Blocker4,Fouler4,ReboundType4,ViolationPlayer4,FreeThrowShooter4,TurnoverPlayer4,ShotDist5,TimeoutTeam5,Substitution5,Shooter5,Rebounder5,Blocker5,Fouler5,ReboundType5,ViolationPlayer5,FreeThrowShooter5,TurnoverPlayer5,ShotDist6,TimeoutTeam6,Substitution6,Shooter6,Rebounder6,Blocker6,Fouler6,ReboundType6,ViolationPlayer6,FreeThrowShooter6,TurnoverPlayer6,ShotDist7,TimeoutTeam7,Substitution7,Shooter7,Rebounder7,Blocker7,Fouler7,ReboundType7,ViolationPlayer7,FreeThrowShooter7,TurnoverPlayer7,ShotDist8,TimeoutTeam8,Substitution8,Shooter8,Rebounder8,Blocker8,Fouler8,ReboundType8,ViolationPlayer8,FreeThrowShooter8,TurnoverPlayer8,ShotDist9,TimeoutTeam9,Substitution9,Shooter9,Rebounder9,Blocker9,Fouler9,ReboundType9,ViolationPlayer9,FreeThrowShooter9,TurnoverPlayer9,ShotDist10,TimeoutTeam10,Substitution10,Shooter10,Rebounder10,Blocker10,Fouler10,ReboundType10,ViolationPlayer10,FreeThrowShooter10,TurnoverPlayer10,class
0,NaN,NaN,NaN,NaN,NaN,NaN,fouler_away,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ft_away,NaN,close,NaN,NaN,shooter_home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mid,NaN,NaN,shooter_away,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rebounder_home,NaN,NaN,defensive,NaN,NaN,NaN,mid,NaN,NaN,shooter_home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rebounder_away,NaN,NaN,defensive,NaN,NaN,NaN,close,NaN,NaN,shooter_away,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,to_player_home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,to_player_away,1
1,NaN,NaN,NaN,NaN,rebounder_away,NaN,NaN,offensive,NaN,NaN,NaN,close,NaN,NaN,shooter_away,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rebounder_away,NaN,NaN,offensive,NaN,NaN,NaN,close,NaN,NaN,shooter_away,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fouler_home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ft_home,NaN,NaN,NaN,NaN,NaN,rebounder_home,NaN,NaN,offensive,NaN,NaN,NaN,NaN,NaN,sub_away,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sub_home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sub_home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,close,NaN,NaN,shooter_home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rebounder_away,NaN,NaN,defensive,NaN,NaN,NaN,3pt,NaN,NaN,shooter_away,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rebounder_away,NaN,NaN,offensive,NaN,NaN,NaN,NaN,NaN,sub_away,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sub_home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3pt,NaN,NaN,shooter_away,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rebounder_home,NaN,NaN,defensive,NaN,NaN,NaN,3pt,NaN,NaN,shooter_home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3pt,NaN,NaN,shooter_away,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,3pt,NaN,NaN,shooter_away,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rebounder_home,NaN,NaN,defensive,NaN,NaN,NaN,NaN,timeout_away,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sub_home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sub_home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fouler_home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fouler_home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ft_home,NaN,NaN,NaN,NaN,NaN,rebounder_home,NaN,NaN,offensive,NaN,NaN,NaN,NaN,NaN,sub_away,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,to_player_away,close,NaN,NaN,shooter_home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,close,NaN,NaN,shooter_away,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3pt,NaN,NaN,shooter_home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mid,NaN,NaN,shooter_away,NaN,NaN,NaN,NaN,NaN,NaN,NaN,close,NaN,NaN,shooter_home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fouler_home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ft_home

In [10]:
combined_df = combined_df.replace({'home': 'same', 'away': 'other'}, regex=True)
combined_df

,ShotDist1,TimeoutTeam1,Substitution1,Shooter1,Rebounder1,Blocker1,Fouler1,ReboundType1,ViolationPlayer1,FreeThrowShooter1,TurnoverPlayer1,ShotDist2,TimeoutTeam2,Substitution2,Shooter2,Rebounder2,Blocker2,Fouler2,ReboundType2,ViolationPlayer2,FreeThrowShooter2,TurnoverPlayer2,ShotDist3,TimeoutTeam3,Substitution3,Shooter3,Rebounder3,Blocker3,Fouler3,ReboundType3,ViolationPlayer3,FreeThrowShooter3,TurnoverPlayer3,ShotDist4,TimeoutTeam4,Substitution4,Shooter4,Rebounder4,Blocker4,Fouler4,ReboundType4,ViolationPlayer4,FreeThrowShooter4,TurnoverPlayer4,ShotDist5,TimeoutTeam5,Substitution5,Shooter5,Rebounder5,Blocker5,Fouler5,ReboundType5,ViolationPlayer5,FreeThrowShooter5,TurnoverPlayer5,ShotDist6,TimeoutTeam6,Substitution6,Shooter6,Rebounder6,Blocker6,Fouler6,ReboundType6,ViolationPlayer6,FreeThrowShooter6,TurnoverPlayer6,ShotDist7,TimeoutTeam7,Substitution7,Shooter7,Rebounder7,Blocker7,Fouler7,ReboundType7,ViolationPlayer7,FreeThrowShooter7,TurnoverPlayer7,ShotDist8,TimeoutTeam8,Substitution8,Shooter8,Rebounder8,Blocker8,Fouler8,ReboundType8,ViolationPlayer8,FreeThrowShooter8,TurnoverPlayer8,ShotDist9,TimeoutTeam9,Substitution9,Shooter9,Rebounder9,Blocker9,Fouler9,ReboundType9,ViolationPlayer9,FreeThrowShooter9,TurnoverPlayer9,ShotDist10,TimeoutTeam10,Substitution10,Shooter10,Rebounder10,Blocker10,Fouler10,ReboundType10,ViolationPlayer10,FreeThrowShooter10,TurnoverPlayer10,class
0,NaN,NaN,NaN,NaN,NaN,NaN,fouler_other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ft_other,NaN,close,NaN,NaN,shooter_same,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mid,NaN,NaN,shooter_other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rebounder_same,NaN,NaN,defensive,NaN,NaN,NaN,mid,NaN,NaN,shooter_same,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rebounder_other,NaN,NaN,defensive,NaN,NaN,NaN,close,NaN,NaN,shooter_other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,to_player_same,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,to_player_other,1
1,NaN,NaN,NaN,NaN,rebounder_other,NaN,NaN,offensive,NaN,NaN,NaN,close,NaN,NaN,shooter_other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rebounder_other,NaN,NaN,offensive,NaN,NaN,NaN,close,NaN,NaN,shooter_other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fouler_same,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ft_same,NaN,NaN,NaN,NaN,NaN,rebounder_same,NaN,NaN,offensive,NaN,NaN,NaN,NaN,NaN,sub_other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sub_same,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sub_same,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,close,NaN,NaN,shooter_same,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rebounder_other,NaN,NaN,defensive,NaN,NaN,NaN,3pt,NaN,NaN,shooter_other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rebounder_other,NaN,NaN,offensive,NaN,NaN,NaN,NaN,NaN,sub_other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sub_same,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3pt,NaN,NaN,shooter_other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rebounder_same,NaN,NaN,defensive,NaN,NaN,NaN,3pt,NaN,NaN,shooter_same,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3pt,NaN,NaN,shooter_other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,3pt,NaN,NaN,shooter_other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rebounder_same,NaN,NaN,defensive,NaN,NaN,NaN,NaN,timeout_other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sub_same,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sub_same,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fouler_same,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fouler_same,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ft_same,NaN,NaN,NaN,NaN,NaN,rebounder_same,NaN,NaN,offensive,NaN,NaN,NaN,NaN,NaN,sub_other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,to_player_other,close,NaN,NaN,shooter_same,NaN,NaN,NaN,NaN,NaN,NaN,NaN,close,NaN,NaN,shooter_other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3pt,NaN,NaN,shooter_same,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mid,NaN,NaN,shooter_other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,close,NaN,NaN,shooter_same,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fouler_same,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [11]:
df = pd.DataFrame()
encoders = []

for column in combined_df.columns[:-1]:
    le = LabelEncoder()
    encoders.append(le)
    df[column] = le.fit_transform(combined_df[column])

df = pd.concat([df,combined_df.iloc[:,-1]], axis=1)

C:\Users\gsevr\AppData\Local\Temp\ipykernel_15352\1916625901.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column] = le.fit_transform(combined_df[column])
C:\Users\gsevr\AppData\Local\Temp\ipykernel_15352\1916625901.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column] = le.fit_transform(combined_df[column])
C:\Users\gsevr\AppData\Local\Temp\ipykernel_15352\1916625901.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perfor

In [12]:
df

,ShotDist1,TimeoutTeam1,Substitution1,Shooter1,Rebounder1,Blocker1,Fouler1,ReboundType1,ViolationPlayer1,FreeThrowShooter1,TurnoverPlayer1,ShotDist2,TimeoutTeam2,Substitution2,Shooter2,Rebounder2,Blocker2,Fouler2,ReboundType2,ViolationPlayer2,FreeThrowShooter2,TurnoverPlayer2,ShotDist3,TimeoutTeam3,Substitution3,Shooter3,Rebounder3,Blocker3,Fouler3,ReboundType3,ViolationPlayer3,FreeThrowShooter3,TurnoverPlayer3,ShotDist4,TimeoutTeam4,Substitution4,Shooter4,Rebounder4,Blocker4,Fouler4,ReboundType4,ViolationPlayer4,FreeThrowShooter4,TurnoverPlayer4,ShotDist5,TimeoutTeam5,Substitution5,Shooter5,Rebounder5,Blocker5,Fouler5,ReboundType5,ViolationPlayer5,FreeThrowShooter5,TurnoverPlayer5,ShotDist6,TimeoutTeam6,Substitution6,Shooter6,Rebounder6,Blocker6,Fouler6,ReboundType6,ViolationPlayer6,FreeThrowShooter6,TurnoverPlayer6,ShotDist7,TimeoutTeam7,Substitution7,Shooter7,Rebounder7,Blocker7,Fouler7,ReboundType7,ViolationPlayer7,FreeThrowShooter7,TurnoverPlayer7,ShotDist8,TimeoutTeam8,Substitution8,Shooter8,Rebounder8,Blocker8,Fouler8,ReboundType8,ViolationPlayer8,FreeThrowShooter8,TurnoverPlayer8,ShotDist9,TimeoutTeam9,Substitution9,Shooter9,Rebounder9,Blocker9,Fouler9,ReboundType9,ViolationPlayer9,FreeThrowShooter9,TurnoverPlayer9,ShotDist10,TimeoutTeam10,Substitution10,Shooter10,Rebounder10,Blocker10,Fouler10,ReboundType10,ViolationPlayer10,FreeThrowShooter10,TurnoverPlayer10,class
0,3,2,2,2,2,2,0,2,2,2,2,3,2,2,2,2,2,2,2,2,0,2,1,2,2,1,2,2,2,2,2,2,2,2,2,2,0,2,2,2,2,2,2,2,3,2,2,2,1,2,2,0,2,2,2,2,2,2,1,2,2,2,2,2,2,2,3,2,2,2,0,2,2,0,2,2,2,1,2,2,0,2,2,2,2,2,2,2,3,2,2,2,2,2,2,2,2,2,1,3,2,2,2,2,2,2,2,2,2,0,1
1,3,2,2,2,0,2,2,1,2,2,2,1,2,2,0,2,2,2,2,2,2,2,3,2,2,2,0,2,2,1,2,2,2,1,2,2,0,2,2,2,2,2,2,2,3,2,2,2,2,2,1,2,2,2,2,3,2,2,2,2,2,2,2,2,1,2,3,2,2,2,1,2,2,1,2,2,2,3,2,0,2,2,2,2,2,2,2,2,3,2,1,2,2,2,2,2,2,2,2,3,2,1,2,2,2,2,2,2,2,2,1
2,1,2,2,1,2,2,2,2,2,2,2,3,2,2,2,0,2,2,0,2,2,2,0,2,2,0,2,2,2,2,2,2,2,3,2,2,2,0,2,2,1,2,2,2,3,2,0,2,2,2,2,2,2,2,2,3,2,1,2,2,2,2,2,2,2,2,0,2,2,0,2,2,2,2,2,2,2,3,2,2,2,1,2,2,0,2,2,2,0,2,2,1,2,2,2,2,2,2,2,0,2,2,0,2,2,2,2,2,2,2,1
3,0,2,2,0,2,2,2,2,2,2,2,3,2,2,2,1,2,2,0,2,2,2,3,0,2,2,2,2,2,2,2,2,2,3,2,1,2,2,2,2,2,2,2,2,3,2,1,2,2,2,2,2,2,2,2,3,2,2,2,2,2,1,2,2,2,2,3,2,2,2,2,2,1,2,2,2,2,3,2,2,2,2,2,2,2,2,1,2,3,2,2,2,1,2,2,1,2,2,2,3,2,0,2,2,2,2,2,2,2,2,1
4,3,2,2,2,2,2,2,2,2,2,0,1,2,2,1,2,2,2,2,2,2,2,1,2,2,0,2,2,2,2,2,2,2,0,2,2,1,2,2,2,2,2,2,2,2,2,2,0,2,2,2,2,2,2,2,1,2,2,1,2,2,2,2,2,2,2,3,2,2,2,2,2,1,2,2,2,2,3,2,2,2,2,2,2,2,2,1,2,3,2,2,2,0,2,2,0,2,2,2,1,2,2,0,2,2,2,2,2,2,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292442,1,2,2,0,2,2,2,2,2,2,2,3,2,2,2,1,2,2,0,2,2,2,0,2,2,1,2,2,2,2,2,2,2,3,2,2,2,1,2,2,1,2,2,2,1,2,2,1,2,2,2,2,2,2,2,3,2,2,2,1,2,2,1,2,2,2,1,2,2,1,2,2,2,2,2,2,2,3,2,2,2,2,2,0,2,2,2,2,3,2,1,2,2,2,2,2,2,2,2,1,2,2,0,2,0,2,2,2,2,2,0
292443,3,2,2,2,0,2,2,1,2,2,2,3,2,2,2,2,2,2,2,2,2,0,1,2,2,1,2,2,2,2,2,2,2,0,2,2,0,2,2,2,2,2,2,2,1,2,2,1,2,2,2,2,2,2,2,0,2,2,0,2,2,2,2,2,2,2,3,2,2,2,1,2,2,0,2,2,2,3,2,2,2,2,2,2,2,2,2,1,3,2,2,2,2,2,2,2,2,2,0,3,2,0,2,2,2,2,2,2,2,2,0
292444,3,2,2,2,2,2,2,2,2,2,1,3,0,2,2,2,2,2,2,2,2,2,1,2,2,0,2,2,2,2,2,2,2,3,2,2,2,2,2,0,2,2,2,2,3,2,2,2,2,2,2,2,2,1,2,3,2,2,2,2,2,1,2,2,2,2,1,2,2,1,2,2,2,2,2,2,2,0,2,2,0,2,2,2,2,2,2,2,3,2,2,2,2,2,1,2,2,2,2,0,2,2,1,2,2,2,2,2,2,2,0
292445,3,2,2,2,2,2,2,2,2,2,0,0,2,2,1,2,2,2,2,2,2,2,3,0,2,2,2,2,2,2,2,2,2,3,2,0,2,2,2,2,2,2,2,2,3,2,0,2,2,2,2,2,2,2,2,3,2,1,2,2,2,2,2,2,2,2,0,2,2,0,2,2,2,2,2,2,2,3,2,2,2,1,2,2,0,2,2,2,3,2,0,2,2,2,2,2,2,2,2,2,2,2,1,2,2,2,2,2,2,2,0


In [13]:
undersample_df = df[df['class'] == 0 ].sample(n=18471,random_state=43)
df = pd.concat([df[df['class'] == 1], undersample_df])

In [14]:
df['class'].value_counts()

class
1    18471
0    18471
Name: count, dtype: int64

In [15]:
import pandas as pd
import time
from collections import defaultdict

run_events = {}
run_max_counts = []

for j, event in zip(range(12,112,11), range(10, 0, -1)):
    a = combined_df.iloc[:, -j:-1][combined_df['class'] == 1]

    # count occurrences of each row
    row_counts = defaultdict(int)
    for i in range(len(a)):
        row_tuple = tuple(a.iloc[i])
        row_counts[row_tuple] += 1

    # row with the maximum count
    max_count = 0
    mc_row = None
    for row, count in row_counts.items():
        if count > max_count:
            max_count = count
            mc_row = row

    # index of the row 
    mc_idx = a.apply(lambda row: tuple(row) == mc_row, axis=1).idxmax()


    print(f"Index: {mc_idx}, Max Count: {max_count}")
    run_events[event] = a.iloc[mc_idx]
    run_max_counts.append(max_count)

Index: 16388, Max Count: 562
Index: 16418, Max Count: 108
Index: 16446, Max Count: 53
Index: 16446, Max Count: 30
Index: 16418, Max Count: 16
Index: 16446, Max Count: 5
Index: 16506, Max Count: 3
Index: 17004, Max Count: 3
Index: 16549, Max Count: 2
Index: 0, Max Count: 1


In [17]:
combined_df.iloc[16418, -23:-1].to_frame().dropna().T

,Rebounder9,ReboundType9,Fouler10
16418,rebounder_same,defensive,fouler_same


In [34]:
import pandas as pd
import time
from collections import defaultdict

run_events = {}
run_max_counts = []

for j, event in zip(range(12,112,11), range(10, 0, -1)):
    a = combined_df.iloc[:, -j:-1][combined_df['class'] == 1]

    # count occurrences of each row in class = 1
    row_counts = defaultdict(int)
    for i in range(len(a)):
        row_tuple = tuple(a.iloc[i])
        row_counts[row_tuple] += 1

    # row with the maximum count
    max_count = 0
    mc_row = None
    for row, count in row_counts.items():
        if count > max_count:
            max_count = count
            mc_row = row

    # index of the row 
    mc_idx = a.apply(lambda row: tuple(row) == mc_row, axis=1).idxmax()

    print(f"Index: {mc_idx}, Max Count: {max_count}")
    run_events[event] = a.iloc[mc_idx]
    run_max_counts.append(max_count)

    # check the number of times this sequence appears in class = 0
    b = combined_df.iloc[:, -j:-1][combined_df['class'] == 0]
    mc_row_count_in_class_0 = b.apply(lambda row: tuple(row) == mc_row, axis=1).sum()
    print(f"Times found in no_run class: {mc_row_count_in_class_0}\n")


Index: 3, Max Count: 4457
Times found in no_run class: 12309

Index: 6, Max Count: 978
Times found in no_run class: 1513

Index: 17, Max Count: 423
Times found in no_run class: 2661

Index: 171, Max Count: 194
Times found in no_run class: 1080

Index: 73, Max Count: 54
Times found in no_run class: 70

Index: 73, Max Count: 24
Times found in no_run class: 25

Index: 153, Max Count: 7
Times found in no_run class: 5

Index: 641, Max Count: 5
Times found in no_run class: 2

Index: 3078, Max Count: 3
Times found in no_run class: 1

Index: 572, Max Count: 2
Times found in no_run class: 0



In [8]:
import pandas as pd
import time
from collections import defaultdict

run_events = {}
run_max_counts = []
ratios = []

for j, event in zip(range(12,112,11), range(10, 0, -1)):
    a = combined_df.iloc[:, -j:-1][combined_df['class'] == 1]

    # count occurrences of each row in class = 1
    row_counts = defaultdict(int)
    for i in range(len(a)):
        row_tuple = tuple(a.iloc[i])
        row_counts[row_tuple] += 1

    # row with the maximum count
    max_count = 0
    mc_row = None
    for row, count in row_counts.items():
        if count > max_count:
            max_count = count
            mc_row = row

    # index of the row 
    mc_idx = a.apply(lambda row: tuple(row) == mc_row, axis=1).idxmax()

    print(f"Index: {mc_idx}, Max Count: {max_count}")
    run_events[event] = a.iloc[mc_idx]
    run_max_counts.append(max_count)

    # check the number of times this sequence appears in class = 0
    b = combined_df.iloc[:, -j:-1][combined_df['class'] == 0]
    mc_row_count_in_class_0 = b.apply(lambda row: tuple(row) == mc_row, axis=1).sum()
    print(f"Times found in no_run class: {mc_row_count_in_class_0}")

    # calculate the ratio
    if mc_row_count_in_class_0 == 0:
        ratio = float('inf')  # or use some large number or a specific value like -1 to indicate infinity
    else:
        ratio = max_count / mc_row_count_in_class_0

    ratios.append(ratio)
    print(f"Ratio: {ratio}")

print(ratios)


Index: 24585, Max Count: 2766
Times found in no_run class: 1584
Ratio: 1.746212121212121
Index: 24594, Max Count: 611
Times found in no_run class: 186
Ratio: 3.28494623655914
Index: 24689, Max Count: 284
Times found in no_run class: 359
Ratio: 0.7910863509749304
Index: 24689, Max Count: 137
Times found in no_run class: 135
Ratio: 1.0148148148148148
Index: 25252, Max Count: 36
Times found in no_run class: 16
Ratio: 2.25
Index: 24978, Max Count: 12
Times found in no_run class: 3
Ratio: 4.0
Index: 24781, Max Count: 6
Times found in no_run class: 1
Ratio: 6.0
Index: 25090, Max Count: 4
Times found in no_run class: 1
Ratio: 4.0
Index: 25236, Max Count: 2
Times found in no_run class: 0
Ratio: inf
Index: 25742, Max Count: 2
Times found in no_run class: 0
Ratio: inf
[1.746212121212121, 3.28494623655914, 0.7910863509749304, 1.0148148148148148, 2.25, 4.0, 6.0, 4.0, inf, inf]
